<a href="https://colab.research.google.com/github/CorsiIsa/Sistema_Recomendacao/blob/master/sistema_recomendacao_als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendação utilizando o algoritmo ALS.

In [16]:
!pip install pyspark

In [17]:
!pip install findspark

Importando a bibliotecas

In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator #evaluation é a biblioteca para verificação da qualidade do modelo
from pyspark.ml.recommendation import ALS # ALS é o modelo de recomendação que será utilizadp
from pyspark.sql import Row #row é o formato que o ALS trabalha, row conterá o id do usuario, id filme, nota e timestamp

Iniciando nossa sessão

In [19]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

Importando nossa base de dados.

Dessa vez iremos trabalhar com arquivos txt.

In [20]:
lines = spark.read.text("sample_movielens_ratings.txt").rdd

Iremos utilizar a função map agora, para cada vez que ele encontrar "::" ele criar as colunas da nossa tabela.

In [21]:
parts = lines.map(lambda row: row.value.split("::"))

Aqui estamos formatando nosso RDD.

Aqui estamos criando os nomes dessas colunas.

In [22]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), \
                                     movieId=int(p[1]), \
                                     rating=float(p[2]), \
                                     timestamp=int(p[3])))

Agora iremos criar um DF com esses dados.

In [23]:
ratings = spark.createDataFrame(ratingsRDD)

In [24]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
+------+-------+------+----------+
only showing top 5 rows



Agora iremos separar nossa base de dados entre treino e teste para o nosso modelo.

In [25]:
(training, test) = ratings.randomSplit([0.8, 0.2])

Agora iremos ajustar o modelo

In [26]:
als = ALS(maxIter=5, \
          regParam=0.01, \
          userCol="userId", \
          itemCol="movieId", \
          ratingCol="rating", \
          coldStartStrategy="drop")

Treinando o modelo.

In [27]:
model = als.fit(training)

In [28]:
predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                               predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print("Erro médio quadrático = " + str(rmse))

Erro médio quadrático = 1.7078012495229986


Iremos pegar todos os usuários e gerar uma lista de recomendações para eles.

In [29]:
userRec = model.recommendForAllUsers(10)

In [30]:
userRec.show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 4.823045}, ...|
|    10|[{2, 3.8207977}, ...|
|     0|[{18, 5.46951}, {...|
|     1|[{46, 5.1202674},...|
|    21|[{53, 4.508073}, ...|
+------+--------------------+
only showing top 5 rows



In [31]:
movieRecs = model.recommendForAllItems(10)

In [32]:
movieRecs.show(5)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{23, 5.1000137},...|
|     40|[{2, 4.3596954}, ...|
|     10|[{23, 3.9383261},...|
|     50|[{12, 4.6172633},...|
|     80|[{0, 4.6006765}, ...|
+-------+--------------------+
only showing top 5 rows



Iremos gerar uma lista de usuários, que irão receber essa recomendação.

In [33]:
users = ratings.select(als.getUserCol()).distinct()

In [34]:
UserRecsOnlyItemId = userRec.select(userRec['userId'], \
                                    userRec['recommendations']['movieid'])

In [35]:
UserRecsOnlyItemId.show(10, False)

+------+----------------------------------------+
|userId|recommendations.movieid                 |
+------+----------------------------------------+
|20    |[22, 52, 77, 46, 75, 51, 18, 94, 80, 88]|
|10    |[2, 40, 53, 89, 25, 68, 88, 49, 74, 92] |
|0     |[18, 88, 80, 4, 92, 41, 70, 24, 9, 52]  |
|1     |[46, 52, 53, 68, 49, 62, 26, 1, 77, 22] |
|21    |[53, 93, 2, 87, 76, 74, 29, 25, 96, 43] |
|11    |[52, 32, 30, 27, 18, 69, 23, 48, 91, 79]|
|12    |[32, 90, 27, 64, 35, 17, 50, 20, 43, 94]|
|22    |[52, 22, 74, 38, 75, 51, 30, 46, 77, 88]|
|2     |[25, 32, 83, 93, 8, 37, 39, 67, 40, 28] |
|13    |[93, 69, 74, 87, 76, 53, 72, 63, 30, 18]|
+------+----------------------------------------+
only showing top 10 rows

